## 📋 Setup & Configuration

First, let's configure the paths and parameters for our evaluation.

In [ ]:
import sys
from pathlib import Path
import json
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration
TEST_IMAGES_DIR = Path("test_data")  # Change this to your test images directory
OUTPUT_DIR = Path("outputs")
CUSTOM_OUTPUT = OUTPUT_DIR / "custom_method"
DIFFEDIT_OUTPUT = OUTPUT_DIR / "diffedit"

# Number of test images to use (set to None to use all)
MAX_TEST_IMAGES = 20  # Adjust based on your needs

# Device configuration
DEVICE = "cuda"  # Change to "cpu" if no GPU available

# Create output directories
CUSTOM_OUTPUT.mkdir(parents=True, exist_ok=True)
DIFFEDIT_OUTPUT.mkdir(parents=True, exist_ok=True)

print(f"✓ Configuration set")
print(f"  Test images: {TEST_IMAGES_DIR}")
print(f"  Output directory: {OUTPUT_DIR}")
print(f"  Device: {DEVICE}")
print(f"  Max images: {MAX_TEST_IMAGES if MAX_TEST_IMAGES else 'All'}")

## 🔍 Understanding the Evaluation Metrics

### How Mask Quality is Evaluated (IoU - Intersection over Union)

**Without Ground Truth Masks:**
Since we don't have manual ground truth masks, we use **cross-validation**:
1. We compare the masks generated by both methods
2. Higher IoU with the original segmentation = better consistency
3. We also evaluate the **quality of the final output** (FID, IS, CLIP)

**The key insight:** A good mask should:
- Accurately segment the vehicle
- Lead to better final image quality
- Be consistent across the pipeline

### Metric Details:

1. **FID (Fréchet Inception Distance)** ⬇️ Lower is Better
   - Compares distribution of generated images to real images
   - Uses deep features from Inception network
   - **Good**: < 50, **Excellent**: < 30
   - Measures: Overall image quality and realism

2. **IS (Inception Score)** ⬆️ Higher is Better
   - Measures quality and diversity of generated images
   - **Good**: > 3.0, **Excellent**: > 4.0
   - Formula: exp(E[KL(p(y|x) || p(y))])
   - High score = diverse, high-quality images

3. **IoU (Intersection over Union)** ⬆️ Higher is Better
   - Measures mask overlap: IoU = (A ∩ B) / (A ∪ B)
   - **Good**: > 0.7, **Excellent**: > 0.85
   - Compares generated masks between methods
   - Higher = more accurate segmentation

4. **LPIPS (Learned Perceptual Image Patch Similarity)** ⬇️ Lower is Better
   - Uses deep learning to measure perceptual similarity
   - **Good**: < 0.3
   - Better than pixel-wise metrics (MSE, SSIM)

5. **CLIP Score** ⬆️ Higher is Better
   - Measures how well image matches text prompt
   - **Good**: > 0.75, **Excellent**: > 0.85
   - Uses CLIP's vision-language alignment

## 📝 Step 1: Prepare Test Images & Prompts

Let's check what test images we have and create prompts for them.

In [ ]:
import os
from PIL import Image

# Check test images
if not TEST_IMAGES_DIR.exists():
    print(f"❌ Test images directory not found: {TEST_IMAGES_DIR}")
    print("Please create it and add test images, or update TEST_IMAGES_DIR")
else:
    image_files = sorted([
        f for f in TEST_IMAGES_DIR.iterdir()
        if f.suffix.lower() in ['.jpg', '.jpeg', '.png']
    ])
    
    if MAX_TEST_IMAGES:
        image_files = image_files[:MAX_TEST_IMAGES]
    
    print(f"✓ Found {len(image_files)} test images")
    
    # Display first few images
    if len(image_files) > 0:
        print("\nFirst few test images:")
        for i, img_file in enumerate(image_files[:5], 1):
            print(f"  {i}. {img_file.name}")
        if len(image_files) > 5:
            print(f"  ... and {len(image_files) - 5} more")

### Create Prompts

You need to create prompts for each test image. Here are two approaches:

**Option A: Use the sample prompts below and customize**
**Option B: Load from existing JSON files**

In [ ]:
# Sample prompts - CUSTOMIZE THESE for your specific images
sample_prompts_custom = {
    # Format: "filename.jpg": {"vehicle": "description", "background": "description"}
}

sample_prompts_diffedit = {
    # Format: "filename.jpg": {"source": "what it is", "target": "what you want"}
}

# Auto-generate generic prompts if you have images but no prompts
# (You should replace these with specific prompts for better results)
if len(image_files) > 0 and len(sample_prompts_custom) == 0:
    print("Creating generic prompts for all images...")
    for img_file in image_files:
        img_name = img_file.name
        # Generic prompts - PLEASE CUSTOMIZE FOR YOUR IMAGES
        sample_prompts_custom[img_name] = {
            "vehicle": "sleek modern sports car",
            "background": "scenic highway at sunset"
        }
        sample_prompts_diffedit[img_name] = {
            "source": "a car on the road",
            "target": "a modern sports car"
        }
    
    print(f"✓ Created generic prompts for {len(image_files)} images")
    print("\n⚠️  WARNING: Using generic prompts!")
    print("   For best results, create specific prompts for each image.")
    print("   Edit the cell above with your custom prompts.")

# Save prompts to files
prompts_custom_file = Path("prompts_custom.json")
prompts_diffedit_file = Path("prompts_diffedit.json")

with open(prompts_custom_file, 'w') as f:
    json.dump(sample_prompts_custom, f, indent=2)

with open(prompts_diffedit_file, 'w') as f:
    json.dump(sample_prompts_diffedit, f, indent=2)

print(f"\n✓ Prompts saved to:")
print(f"  - {prompts_custom_file}")
print(f"  - {prompts_diffedit_file}")

## 🚀 Step 2: Run Custom Method

This will process all test images through your 4-stage pipeline:
1. **Stage 1**: UNet segmentation (original image)
2. **Stage 2**: Stable Diffusion vehicle regeneration
3. **Stage 3**: UNet re-segmentation (edited image)
4. **Stage 4**: Stable Diffusion background inpainting

In [ ]:
# Import custom pipeline functions
from main import (
    segment_image,
    regenerate_vehicle,
    segment_edited_image,
    inpaint_background,
    BASE_DIR
)

# Model paths
unet_path = BASE_DIR / "model" / "unet_model_carvana_new.pth"
sd_model_path = (
    BASE_DIR / "model" / "stable-diffusion" / 
    "models--runwayml--stable-diffusion-v1-5" / "snapshots" /
    "451f4fe16113bff5a5d2269ed5ad43b0592e9a14"
)

# Check models exist
if not unet_path.exists():
    print(f"❌ UNet model not found: {unet_path}")
    print("Please run setup.py to download models")
else:
    print(f"✓ UNet model found")

if not sd_model_path.exists():
    print(f"❌ Stable Diffusion model not found: {sd_model_path}")
    print("Please run setup.py to download models")
else:
    print(f"✓ Stable Diffusion model found")

In [ ]:
# Process images with custom method
print("Processing images with Custom Method...")
print("="*60)

custom_results = []

for img_file in tqdm(image_files, desc="Custom Method"):
    img_name = img_file.name
    
    if img_name not in sample_prompts_custom:
        print(f"⚠ No prompts for {img_name}, skipping...")
        continue
    
    prompts = sample_prompts_custom[img_name]
    vehicle_prompt = prompts.get('vehicle', '')
    background_prompt = prompts.get('background', '')
    
    try:
        # Stage 1: Initial Segmentation
        stage1_mask_name = f"stage1_mask_{img_name}"
        stage1_mask_path = segment_image(
            img_path=str(img_file),
            model_path=str(unet_path),
            output_dir=CUSTOM_OUTPUT,
            output_name=stage1_mask_name
        )
        
        # Stage 2: Vehicle Regeneration
        stage2_vehicle_name = f"stage2_vehicle_{img_name}"
        _, stage2_vehicle_path = regenerate_vehicle(
            img_path=str(img_file),
            mask_path=stage1_mask_path,
            model_dir=str(sd_model_path),
            prompt=vehicle_prompt,
            output_dir=CUSTOM_OUTPUT,
            output_name=stage2_vehicle_name
        )
        
        # Stage 3: Re-segmentation
        stage3_mask_name = f"stage3_mask_{img_name}"
        stage3_mask_path = segment_edited_image(
            img_path=stage2_vehicle_path,
            model_path=str(unet_path),
            output_dir=CUSTOM_OUTPUT,
            output_name=stage3_mask_name
        )
        
        # Stage 4: Background Inpainting
        stage4_final_name = f"final_{img_name}"
        _, stage4_final_path = inpaint_background(
            img_path=stage2_vehicle_path,
            mask_path=stage3_mask_path,
            model_dir=str(sd_model_path),
            prompt=background_prompt,
            output_dir=CUSTOM_OUTPUT,
            output_name=stage4_final_name
        )
        
        custom_results.append({
            'image': img_name,
            'status': 'success',
            'stage1_mask': stage1_mask_path,
            'stage3_mask': stage3_mask_path,
            'final': stage4_final_path
        })
        
    except Exception as e:
        print(f"\n❌ Error processing {img_name}: {e}")
        custom_results.append({
            'image': img_name,
            'status': 'error',
            'error': str(e)
        })

successful_custom = len([r for r in custom_results if r['status'] == 'success'])
print(f"\n✓ Custom method completed: {successful_custom}/{len(image_files)} images successful")

## 🎨 Step 3: Run DiffEdit

Now let's run DiffEdit on the same images for comparison.

In [ ]:
import torch
from diffusers import DDIMScheduler, DDIMInverseScheduler, StableDiffusionDiffEditPipeline
from diffusers.utils import load_image

# Setup DiffEdit pipeline
print("Loading DiffEdit pipeline...")
diffedit_pipeline = StableDiffusionDiffEditPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    torch_dtype=torch.float16,
    safety_checker=None,
    use_safetensors=True,
)

diffedit_pipeline.scheduler = DDIMScheduler.from_config(diffedit_pipeline.scheduler.config)
diffedit_pipeline.inverse_scheduler = DDIMInverseScheduler.from_config(
    diffedit_pipeline.scheduler.config
)

diffedit_pipeline.enable_model_cpu_offload()
diffedit_pipeline.enable_vae_slicing()

print("✓ DiffEdit pipeline loaded")

In [ ]:
# Process images with DiffEdit
print("\nProcessing images with DiffEdit...")
print("="*60)
print("Note: DiffEdit includes an inversion step, so each image takes longer (~60-120s)")

diffedit_results = []
IMAGE_SIZE = (768, 768)

for img_file in tqdm(image_files, desc="DiffEdit"):
    img_name = img_file.name
    
    if img_name not in sample_prompts_diffedit:
        print(f"⚠ No prompts for {img_name}, skipping...")
        continue
    
    prompts = sample_prompts_diffedit[img_name]
    source_prompt = prompts.get('source', '')
    target_prompt = prompts.get('target', '')
    
    try:
        # Load and resize image
        raw_image = load_image(str(img_file)).resize(IMAGE_SIZE)
        
        # Generate mask
        mask_image = diffedit_pipeline.generate_mask(
            image=raw_image,
            source_prompt=source_prompt,
            target_prompt=target_prompt,
        )
        
        # Invert latents
        inv_latents = diffedit_pipeline.invert(
            prompt=source_prompt,
            image=raw_image
        ).latents
        
        # Generate final image
        output_image = diffedit_pipeline(
            prompt=target_prompt,
            mask_image=mask_image,
            image_latents=inv_latents,
            negative_prompt=source_prompt,
        ).images[0]
        
        # Save outputs
        output_path = DIFFEDIT_OUTPUT / f"edited_{img_name}"
        output_image.save(output_path)
        
        # Save mask
        mask_pil = Image.fromarray((mask_image.squeeze()*255).astype("uint8"), "L")
        mask_pil = mask_pil.resize(IMAGE_SIZE)
        mask_path = DIFFEDIT_OUTPUT / f"mask_{img_name}"
        mask_pil.save(mask_path)
        
        diffedit_results.append({
            'image': img_name,
            'status': 'success',
            'output': str(output_path),
            'mask': str(mask_path)
        })
        
    except Exception as e:
        print(f"\n❌ Error processing {img_name}: {e}")
        diffedit_results.append({
            'image': img_name,
            'status': 'error',
            'error': str(e)
        })

successful_diffedit = len([r for r in diffedit_results if r['status'] == 'success'])
print(f"\n✓ DiffEdit completed: {successful_diffedit}/{len(image_files)} images successful")

# Free memory
del diffedit_pipeline
torch.cuda.empty_cache()

## 📊 Step 4: Compute Evaluation Metrics

Now let's compute all the metrics to compare both methods.

In [ ]:
import torch
import numpy as np
from torchmetrics.image.fid import FrechetInceptionDistance
from torchmetrics.image.lpip import LearnedPerceptualImagePatchSimilarity
import clip
from torchvision import transforms
from scipy.stats import entropy
import torch.nn.functional as F
from torchvision.models import inception_v3

print("Initializing evaluation metrics...")

# Initialize metrics
fid_metric = FrechetInceptionDistance(normalize=True).to(DEVICE)
lpips_metric = LearnedPerceptualImagePatchSimilarity(net_type='alex').to(DEVICE)
clip_model, clip_preprocess = clip.load("ViT-B/32", device=DEVICE)

print("✓ Metrics initialized")

In [ ]:
# Helper functions for metric computation

def preprocess_for_fid(image_path):
    """Convert image to tensor for FID"""
    img = Image.open(image_path).convert("RGB")
    img_array = np.array(img)
    if img_array.dtype != np.uint8:
        img_array = (img_array * 255).astype(np.uint8)
    tensor = torch.from_numpy(img_array).permute(2, 0, 1).unsqueeze(0)
    return tensor.to(DEVICE)

def preprocess_for_lpips(image_path):
    """Convert image to tensor for LPIPS"""
    img = Image.open(image_path).convert("RGB")
    transform = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
    ])
    return transform(img).unsqueeze(0).to(DEVICE)

def calculate_clip_similarity(image_path, text_prompt):
    """Calculate CLIP image-text similarity"""
    img = Image.open(image_path).convert("RGB")
    image_input = clip_preprocess(img).unsqueeze(0).to(DEVICE)
    text_input = clip.tokenize([text_prompt]).to(DEVICE)
    
    with torch.no_grad():
        image_features = clip_model.encode_image(image_input)
        text_features = clip_model.encode_text(text_input)
        
        image_features = image_features / image_features.norm(dim=-1, keepdim=True)
        text_features = text_features / text_features.norm(dim=-1, keepdim=True)
        
        similarity = (image_features @ text_features.T).item()
    
    return similarity

def calculate_iou(mask1_path, mask2_path):
    """Calculate IoU between two masks"""
    mask1 = np.array(Image.open(mask1_path).convert("L")) > 127
    mask2 = np.array(Image.open(mask2_path).convert("L")) > 127
    
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    
    if union == 0:
        return 0.0
    return intersection / union

print("✓ Helper functions defined")

In [ ]:
# Compute metrics for Custom Method
print("Computing metrics for Custom Method...")
print("="*60)

custom_metrics = {
    'lpips_scores': [],
    'clip_scores': [],
    'iou_scores': [],
    'per_image': []
}

# Reset FID
fid_metric.reset()

for result in tqdm(custom_results, desc="Custom Metrics"):
    if result['status'] != 'success':
        continue
    
    img_name = result['image']
    orig_path = TEST_IMAGES_DIR / img_name
    final_path = Path(result['final'])
    
    # FID
    orig_tensor = preprocess_for_fid(orig_path)
    final_tensor = preprocess_for_fid(final_path)
    fid_metric.update(orig_tensor, real=True)
    fid_metric.update(final_tensor, real=False)
    
    # LPIPS
    orig_lpips = preprocess_for_lpips(orig_path)
    final_lpips = preprocess_for_lpips(final_path)
    lpips_score = lpips_metric(final_lpips, orig_lpips).item()
    custom_metrics['lpips_scores'].append(lpips_score)
    
    # CLIP (using target prompt)
    if img_name in sample_prompts_custom:
        prompts = sample_prompts_custom[img_name]
        combined_prompt = f"{prompts['vehicle']} on {prompts['background']}"
        clip_score = calculate_clip_similarity(final_path, combined_prompt)
        custom_metrics['clip_scores'].append(clip_score)
    
    # IoU (comparing stage 1 and stage 3 masks for consistency)
    iou_score = calculate_iou(result['stage1_mask'], result['stage3_mask'])
    custom_metrics['iou_scores'].append(iou_score)
    
    custom_metrics['per_image'].append({
        'image': img_name,
        'lpips': lpips_score,
        'clip': clip_score if img_name in sample_prompts_custom else None,
        'iou': iou_score
    })

# Compute FID
custom_metrics['fid'] = fid_metric.compute().item()
custom_metrics['avg_lpips'] = np.mean(custom_metrics['lpips_scores'])
custom_metrics['avg_clip'] = np.mean(custom_metrics['clip_scores'])
custom_metrics['avg_iou'] = np.mean(custom_metrics['iou_scores'])

print(f"✓ Custom Method Metrics:")
print(f"  FID:   {custom_metrics['fid']:.3f}")
print(f"  LPIPS: {custom_metrics['avg_lpips']:.4f}")
print(f"  CLIP:  {custom_metrics['avg_clip']:.4f}")
print(f"  IoU:   {custom_metrics['avg_iou']:.4f}")

In [ ]:
# Compute metrics for DiffEdit
print("\nComputing metrics for DiffEdit...")
print("="*60)

diffedit_metrics = {
    'lpips_scores': [],
    'clip_scores': [],
    'iou_scores': [],
    'per_image': []
}

# Reset FID
fid_metric.reset()

for result in tqdm(diffedit_results, desc="DiffEdit Metrics"):
    if result['status'] != 'success':
        continue
    
    img_name = result['image']
    orig_path = TEST_IMAGES_DIR / img_name
    output_path = Path(result['output'])
    
    # FID
    orig_tensor = preprocess_for_fid(orig_path)
    output_tensor = preprocess_for_fid(output_path)
    fid_metric.update(orig_tensor, real=True)
    fid_metric.update(output_tensor, real=False)
    
    # LPIPS
    orig_lpips = preprocess_for_lpips(orig_path)
    output_lpips = preprocess_for_lpips(output_path)
    lpips_score = lpips_metric(output_lpips, orig_lpips).item()
    diffedit_metrics['lpips_scores'].append(lpips_score)
    
    # CLIP
    if img_name in sample_prompts_diffedit:
        target_prompt = sample_prompts_diffedit[img_name]['target']
        clip_score = calculate_clip_similarity(output_path, target_prompt)
        diffedit_metrics['clip_scores'].append(clip_score)
    
    # IoU (compare with custom method's mask for the same image)
    custom_result = next((r for r in custom_results if r['image'] == img_name), None)
    if custom_result and custom_result['status'] == 'success':
        iou_score = calculate_iou(result['mask'], custom_result['stage3_mask'])
        diffedit_metrics['iou_scores'].append(iou_score)
    
    diffedit_metrics['per_image'].append({
        'image': img_name,
        'lpips': lpips_score,
        'clip': clip_score if img_name in sample_prompts_diffedit else None,
        'iou': iou_score if custom_result else None
    })

# Compute FID
diffedit_metrics['fid'] = fid_metric.compute().item()
diffedit_metrics['avg_lpips'] = np.mean(diffedit_metrics['lpips_scores'])
diffedit_metrics['avg_clip'] = np.mean(diffedit_metrics['clip_scores'])
diffedit_metrics['avg_iou'] = np.mean(diffedit_metrics['iou_scores']) if diffedit_metrics['iou_scores'] else 0

print(f"✓ DiffEdit Metrics:")
print(f"  FID:   {diffedit_metrics['fid']:.3f}")
print(f"  LPIPS: {diffedit_metrics['avg_lpips']:.4f}")
print(f"  CLIP:  {diffedit_metrics['avg_clip']:.4f}")
print(f"  IoU:   {diffedit_metrics['avg_iou']:.4f}")

## 📈 Step 5: Compare Results

Let's create a comprehensive comparison of both methods.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

# Create comparison table
print("\n" + "="*70)
print("COMPARATIVE RESULTS: Custom Method vs DiffEdit")
print("="*70 + "\n")

comparison_data = {
    'Metric': ['FID ↓', 'LPIPS ↓', 'CLIP ↑', 'IoU ↑'],
    'Custom Method': [
        f"{custom_metrics['fid']:.3f}",
        f"{custom_metrics['avg_lpips']:.4f}",
        f"{custom_metrics['avg_clip']:.4f}",
        f"{custom_metrics['avg_iou']:.4f}"
    ],
    'DiffEdit': [
        f"{diffedit_metrics['fid']:.3f}",
        f"{diffedit_metrics['avg_lpips']:.4f}",
        f"{diffedit_metrics['avg_clip']:.4f}",
        f"{diffedit_metrics['avg_iou']:.4f}"
    ]
}

df = pd.DataFrame(comparison_data)

# Determine winners
winners = []
metrics_list = [
    ('fid', False),  # lower is better
    ('avg_lpips', False),
    ('avg_clip', True),  # higher is better
    ('avg_iou', True)
]

for metric_key, higher_better in metrics_list:
    custom_val = custom_metrics[metric_key]
    diffedit_val = diffedit_metrics[metric_key]
    
    if higher_better:
        winner = '✓ Custom' if custom_val > diffedit_val else '✓ DiffEdit'
    else:
        winner = '✓ Custom' if custom_val < diffedit_val else '✓ DiffEdit'
    
    winners.append(winner)

df['Winner'] = winners

print(df.to_string(index=False))
print("\n" + "="*70)

# Count wins
custom_wins = winners.count('✓ Custom')
diffedit_wins = winners.count('✓ DiffEdit')

print(f"\nOVERALL WINNER: ", end="")
if custom_wins > diffedit_wins:
    print("✓ Custom Method")
elif diffedit_wins > custom_wins:
    print("✓ DiffEdit")
else:
    print("Tie")

print(f"(Custom: {custom_wins} wins, DiffEdit: {diffedit_wins} wins)")
print("="*70 + "\n")

## 📊 Step 6: Visualize Results

Let's create visual comparisons of the methods.

In [ ]:
# Create bar chart comparison
fig, axes = plt.subplots(1, 4, figsize=(20, 5))

metrics_to_plot = [
    ('FID', 'fid', False),
    ('LPIPS', 'avg_lpips', False),
    ('CLIP', 'avg_clip', True),
    ('IoU', 'avg_iou', True)
]

for idx, (name, key, higher_better) in enumerate(metrics_to_plot):
    custom_val = custom_metrics[key]
    diffedit_val = diffedit_metrics[key]
    
    # Determine colors
    if higher_better:
        colors = ['green' if custom_val > diffedit_val else 'lightgreen',
                 'blue' if diffedit_val > custom_val else 'lightblue']
    else:
        colors = ['green' if custom_val < diffedit_val else 'lightgreen',
                 'blue' if diffedit_val < custom_val else 'lightblue']
    
    bars = axes[idx].bar(['Custom', 'DiffEdit'], 
                        [custom_val, diffedit_val],
                        color=colors)
    
    # Add value labels
    for bar in bars:
        height = bar.get_height()
        axes[idx].text(bar.get_x() + bar.get_width()/2., height,
                      f'{height:.3f}',
                      ha='center', va='bottom', fontsize=11)
    
    # Styling
    direction = '↑ Higher Better' if higher_better else '↓ Lower Better'
    axes[idx].set_title(f'{name}\n{direction}', fontsize=13, fontweight='bold')
    axes[idx].set_ylabel('Score', fontsize=11)
    axes[idx].grid(axis='y', alpha=0.3)

plt.suptitle('Metrics Comparison: Custom Method vs DiffEdit', 
             fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'metrics_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Metrics chart saved to: {OUTPUT_DIR / 'metrics_comparison.png'}")

In [ ]:
# Create side-by-side image comparisons
print("\nCreating visual comparisons of individual images...")

# Select up to 5 images to display
display_images = min(5, len([r for r in custom_results if r['status'] == 'success']))

fig, axes = plt.subplots(display_images, 3, figsize=(15, 5 * display_images))

if display_images == 1:
    axes = axes.reshape(1, -1)

success_results = [(c, d) for c, d in zip(custom_results, diffedit_results) 
                   if c['status'] == 'success' and d['status'] == 'success']

for idx, (custom_result, diffedit_result) in enumerate(success_results[:display_images]):
    img_name = custom_result['image']
    
    # Load images
    original = Image.open(TEST_IMAGES_DIR / img_name)
    custom_output = Image.open(custom_result['final'])
    diffedit_output = Image.open(diffedit_result['output'])
    
    # Display
    axes[idx, 0].imshow(original)
    axes[idx, 0].set_title(f'Original\n{img_name}', fontsize=10)
    axes[idx, 0].axis('off')
    
    axes[idx, 1].imshow(custom_output)
    axes[idx, 1].set_title('Custom Method', fontsize=10, color='green', fontweight='bold')
    axes[idx, 1].axis('off')
    
    axes[idx, 2].imshow(diffedit_output)
    axes[idx, 2].set_title('DiffEdit', fontsize=10, color='blue', fontweight='bold')
    axes[idx, 2].axis('off')

plt.suptitle('Side-by-Side Comparison of Sample Images', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig(OUTPUT_DIR / 'visual_comparison.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"✓ Visual comparison saved to: {OUTPUT_DIR / 'visual_comparison.png'}")

## 💾 Step 7: Save Results

Let's save all results to a JSON file for future reference.

In [ ]:
from datetime import datetime

# Compile all results
final_results = {
    'timestamp': datetime.now().isoformat(),
    'test_configuration': {
        'num_images': len(image_files),
        'device': DEVICE,
        'test_images_dir': str(TEST_IMAGES_DIR)
    },
    'custom_method': {
        'fid': custom_metrics['fid'],
        'lpips': custom_metrics['avg_lpips'],
        'clip': custom_metrics['avg_clip'],
        'iou': custom_metrics['avg_iou'],
        'num_successful': len([r for r in custom_results if r['status'] == 'success'])
    },
    'diffedit_method': {
        'fid': diffedit_metrics['fid'],
        'lpips': diffedit_metrics['avg_lpips'],
        'clip': diffedit_metrics['avg_clip'],
        'iou': diffedit_metrics['avg_iou'],
        'num_successful': len([r for r in diffedit_results if r['status'] == 'success'])
    },
    'per_image_comparison': []
}

# Add per-image details
for custom_img, diffedit_img in zip(custom_metrics['per_image'], diffedit_metrics['per_image']):
    final_results['per_image_comparison'].append({
        'image': custom_img['image'],
        'custom': custom_img,
        'diffedit': diffedit_img
    })

# Save to file
results_file = OUTPUT_DIR / 'evaluation_results.json'
with open(results_file, 'w') as f:
    json.dump(final_results, f, indent=2)

print(f"✓ Results saved to: {results_file}")

## 🎯 Summary & Interpretation

### Key Takeaways

Your custom method should outperform DiffEdit if:
- **FID is lower** → Better image quality
- **IoU is higher** → Better segmentation accuracy
- **CLIP is higher** → Better text-prompt alignment

### What the IoU Score Actually Means

Since we don't have ground truth masks, the IoU here measures:
1. **For Custom Method**: Consistency between Stage 1 and Stage 3 masks
   - High IoU = the segmentation is stable/consistent
2. **Between Methods**: How similar the masks are
   - This shows if both methods identify similar regions

### Understanding "Good" vs "Bad" Masks

A good mask:
- ✓ Cleanly separates foreground from background
- ✓ Follows object boundaries accurately
- ✓ Leads to better final image quality (reflected in FID/CLIP)
- ✓ Is consistent across the pipeline

A bad mask:
- ✗ Has rough/jagged edges
- ✗ Misses parts of the object or includes too much background
- ✗ Causes artifacts in the final image
- ✗ Is inconsistent between stages

### Why Custom Method Should Win

Your 4-stage approach has advantages:
1. **Trained segmentation** (UNet) vs heuristic mask generation
2. **Sequential refinement** (re-segment after editing)
3. **Specialized control** (vehicle and background separately)

### Next Steps

1. Review the visual comparisons above
2. Check the JSON file for detailed per-image metrics
3. Identify failure cases (if any)
4. Fine-tune prompts for better results
5. Use these results in your report/paper

## 📝 Recommended Test Set Sizes

Based on your evaluation needs:

| Purpose | Images | Time (GPU) | Why |
|---------|--------|------------|-----|
| **Quick Test** | 5-10 | ~15-30 min | Verify pipeline works |
| **Development** | 10-15 | ~30-45 min | Iterate on prompts |
| **Evaluation** | 20-30 | ~1-2 hours | Reliable statistics |
| **Publication** | 50-100 | ~3-5 hours | Publication-quality results |

### Statistical Reliability

- **< 10 images**: Results may be unreliable, high variance
- **10-20 images**: Moderate confidence, good for class projects
- **20-30 images**: Good confidence, suitable for papers
- **50+ images**: High confidence, publication-quality

The metrics become more stable with more images, especially FID and IS which measure distribution similarity.

## ✅ Evaluation Complete!

You've successfully:
- ✓ Run both methods on your test images
- ✓ Computed quantitative metrics (FID, LPIPS, CLIP, IoU)
- ✓ Created visual comparisons
- ✓ Saved all results to JSON

Check the `outputs/` directory for all generated images and the results JSON file.